In [13]:
from pathlib import Path
if Path("../data/").exists():
    print("data already downloaded")
    pass
else:
    !gsutil cp -r gs://translation-dataset-aa/data ../

data already downloaded


In [14]:
from translation_machine import dataset_mod
import numpy as np,torch,os,itertools
from torchtext.data.utils import get_tokenizer
# we use the same tokenizer for english and 
english_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
french_tokenizer = get_tokenizer('spacy', language='fr_core_news_sm')

In [15]:
from itertools import islice
import torchtext
from torchtext.vocab import build_vocab_from_iterator



english_iterator = (english_tokenizer(english_sentence) for english_sentence,french_sentence in train_val_dataset)
french_iterator = (french_tokenizer(french_sentence) for english_sentence,french_sentence in train_val_dataset)

vocab_english = torchtext.vocab.build_vocab_from_iterator(english_iterator,specials=['<unk>'])
vocab_french = torchtext.vocab.build_vocab_from_iterator(french_iterator,specials=['<unk>','<sos>','<eos>'])

vocab_english.set_default_index(vocab_english['<unk>'])
vocab_french.set_default_index(vocab_french['<unk>'])
len(vocab_english),len(vocab_french)

(12, 19)

In [18]:
#getting the tallest sequences for each language
length_en_sentences = []
length_fr_sentrences = []
for el in train_val_dataset:
    length_en_sentences.append(len(el[0]))
    length_fr_sentrences.append(len(el[1]))
max_length_french = max(length_fr_sentrences)
max_length_english = max(length_en_sentences)
max_length_english,max_length_french

(5, 9)

### saving the vocabulary along with the model

In [19]:
import os
os.makedirs("../models/vocabs",exist_ok=True)

In [20]:
french_language_info = {"vocab":vocab_french,
                           "max_sentence_train_val":max_length_french}
english_language_info = {"vocab":vocab_english,
                           "max_sentence_train_val":max_length_english}

language_info = {"french":french_language_info,
                "english":english_language_info,
                "length_dataset_for_vocab":len(train_val_dataset)}

torch.save(language_info,path_language_info)

### some  sanity tests for io of vocabs  

In [21]:
language_info_from_file = torch.load(path_language_info)
vocab_english_from_file = language_info["english"]["vocab"]
vocab_french_from_file = language_info["french"]["vocab"]

In [22]:
assert vocab_french.vocab.get_itos() == vocab_french_from_file.vocab.get_itos() 
assert vocab_french.vocab.get_stoi() == vocab_french_from_file.vocab.get_stoi() 

In [23]:
assert vocab_english.vocab.get_itos() == vocab_english.vocab.get_itos()
assert vocab_english.vocab.get_stoi() == vocab_english_from_file.vocab.get_stoi() 

In [25]:
vocab_english.vocab.get_itos(),vocab_english.vocab.get_itos()

(['<unk>',
  '!',
  '.',
  'Hi',
  'Run',
  '?',
  'Fire',
  'Go',
  'Help',
  'Jump',
  'Who',
  'Wow'],
 ['<unk>',
  '!',
  '.',
  'Hi',
  'Run',
  '?',
  'Fire',
  'Go',
  'Help',
  'Jump',
  'Who',
  'Wow'])